# Kaggle Comp Jigsaw Rules Classification

### Dependencies

In [7]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import os
import re

### Preparing the Test Data

In [8]:
test_df = pd.read_csv(os.path.join("Data/", 'test.csv'))
test_df.set_index('row_id', inplace=True)
test_df.head()

,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
row_id,,,,,,,
2029,NEW RAP GROUP 17. CHECK US OUT https://soundcl...,"No Advertising: Spam, referral links, unsolici...",hiphopheads,"Hey, guys, just wanted to drop in and invite y...",Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng - [Chelsea TV USA](http://soccer...,HD Streams: |[ENG HD Stoke vs Manchester Unite...
2030,Make your life comfortable. Get up to 15% Disc...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn't drastic. You tried reaching out to ...,So what are you going to do with the insurance...,It's just for Austria & Germany. If you still ...
2031,Kickin' ass and selling underwear!\nJust made ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend. Check out my blog at ht...,If you know what exactly you need then you don...,CENTIPEDES\n\nSOME BASED PATRIOTS HAVE CREATED...,[So great! Thanks for sharing.](http://www.che...
2032,watch hooters best therein http://clickan...,"No Advertising: Spam, referral links, unsolici...",personalfinance,"Earn 50,000 bonus points with Chase Sapphire P...","Cool, front page! I made this print along with...",[Full HD Movie Online Free](http://www.flickma...,* Karambit Black Pearl\n* 0.02137822 Float (un...
2033,bitches for free at this point show all h...,"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande --->>> [Imgur](http://i.imgu...,My trade link\nhttps://steamcommunity.com/trad...,**HD** [ mio Stadium 102 HD](http://www.genti....,Infographics is an incredible method for showi...


In [9]:
test_df.isnull().sum()

body                  0
rule                  0
subreddit             0
positive_example_1    0
positive_example_2    0
negative_example_1    0
negative_example_2    0
dtype: int64

In [10]:

test_df["body"] = test_df["body"].map(lambda x: re.sub(r"http[s]?://\S+", "[LINK]", x))
test_df["positive_example_1"] = test_df["positive_example_1"].map(lambda x: re.sub(r"http[s]?://\S+", "[LINK]", x))
test_df["positive_example_2"] = test_df["positive_example_2"].map(lambda x: re.sub(r"http[s]?://\S+", "[LINK]", x))
test_df["negative_example_1"] = test_df["negative_example_1"].map(lambda x: re.sub(r"http[s]?://\S+", "[LINK]", x))
test_df["negative_example_2"] = test_df["negative_example_2"].map(lambda x: re.sub(r"http[s]?://\S+", "[LINK]", x))

test_df["body"] = test_df["body"].map(lambda x: re.sub(r"[^\s\w#\]\[]", " ", x))
test_df["positive_example_1"] = test_df["positive_example_1"].map(lambda x: re.sub(r"[^\s\w#\]\[]", " ", x))
test_df["positive_example_2"] = test_df["positive_example_2"].map(lambda x: re.sub(r"[^\s\w#\]\[]", " ", x))
test_df["negative_example_1"] = test_df["negative_example_1"].map(lambda x: re.sub(r"[^\s\w#\]\[]", " ", x))
test_df["negative_example_2"] = test_df["negative_example_2"].map(lambda x: re.sub(r"[^\s\w#\]\[]", " ", x))

test_df["body"] = test_df["body"].map(lambda x: re.sub(r"\s\s+", " ", x))
test_df["positive_example_1"] = test_df["positive_example_1"].map(lambda x: re.sub(r"\s\s+", " ", x))
test_df["positive_example_2"] = test_df["positive_example_2"].map(lambda x: re.sub(r"\s\s+", " ", x))
test_df["negative_example_1"] = test_df["negative_example_1"].map(lambda x: re.sub(r"\s\s+", " ", x))
test_df["negative_example_2"] = test_df["negative_example_2"].map(lambda x: re.sub(r"\s\s+", " ", x))

test_df.head()

,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
row_id,,,,,,,
2029,NEW RAP GROUP 17 CHECK US OUT [LINK],"No Advertising: Spam, referral links, unsolici...",hiphopheads,Hey guys just wanted to drop in and invite you...,Cum Swallowing Hottie Katrina Kaif Cartoon Xvi...,SD Stream Eng [Chelsea TV USA] [LINK] Mobile C...,HD Streams [ENG HD Stoke vs Manchester United]...
2030,Make your life comfortable Get up to 15 Discou...,No legal advice: Do not offer or request legal...,AskReddit,Get a lawyer and get the security camera foota...,That isn t drastic You tried reaching out to h...,So what are you going to do with the insurance...,It s just for Austria Germany If you still nee...
2031,Kickin ass and selling underwear Just made my ...,"No Advertising: Spam, referral links, unsolici...",gonewild,Good story my friend Check out my blog at [LINK],If you know what exactly you need then you don...,CENTIPEDES SOME BASED PATRIOTS HAVE CREATED A ...,[So great Thanks for sharing ] [LINK]
2032,watch hooters best therein [LINK],"No Advertising: Spam, referral links, unsolici...",personalfinance,Earn 50 000 bonus points with Chase Sapphire P...,Cool front page I made this print along with a...,[Full HD Movie Online Free] [LINK],Karambit Black Pearl 0 02137822 Float unfortu...
2033,bitches for free at this point show all [LINK],"No Advertising: Spam, referral links, unsolici...",Showerthoughts,code free tyrande [Imgur] [LINK] for you and y...,My trade link\n[LINK],HD [ mio Stadium 102 HD] [LINK] Mobile Compat...,Infographics is an incredible method for showi...


### Preaparing the sample submission 

In [13]:
sample = pd.read_csv(os.path.join("Data","sample_submission.csv"))
sample.set_index("row_id",inplace = True)
sample.head()

,rule_violation
row_id,
2029,0.5
2030,0.5
2031,0.5
2032,0.5
2033,0.5


### MODEL: cvlr-o1

In [18]:
X = test_df[["body", "rule","subreddit","positive_example_1","positive_example_2","negative_example_1","negative_example_2"]].astype(str).agg(' '.join, axis=1)

In [21]:
model = joblib.load(os.path.join("TrainedModels","cvlr-o1.bin"))
yhat = model.predict(X)
sample["rule_violation"] = yhat
sample.head()

,rule_violation
row_id,
2029,0
2030,0
2031,1
2032,1
2033,1


In [22]:
sample.to_csv(os.path.join("TrainedModels","cvlr-o1.csv"))